In [ ]:
import pathlib as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import simple_soil
import flopy.plot.styles as styles

Maximum infiltration

```
(upslope_hortonian + rain + snowmelt - hru_sroffi - hru_sroffp) x (1-hru_percent_imperv)
```

Conversion factors

In [ ]:
in2m = 1.0 / (12.0 * 3.28081)
acre2sqm = 4046.86

In [ ]:
hru_percent_imperv = 0.309456

Forcing data

In [ ]:
data_dir = pl.Path("data/hru_1")

In [ ]:
tag = " 2"

In [ ]:
# rainfall
rain = pd.read_csv(data_dir / "hru_rain.csv", index_col=0)
snow = pd.read_csv(data_dir / "snowmelt.csv", index_col=0)
sroffi = pd.read_csv(data_dir / "hru_sroffi.csv", index_col=0)
sroffp = pd.read_csv(data_dir / "hru_sroffi.csv", index_col=0)
rain["ppt m/day"] = (
    (rain[tag] + snow[tag] - sroffi[tag] - sroffp[tag])
    * (1.0 - hru_percent_imperv)
    * in2m
)
rain = rain.drop(tag, axis=1)
rain.columns

In [ ]:
rain.plot()

Soil potential evapotranspiration

```
potet - hru_intcpevap - snow_evap - hru_impervevap - dprst_evap_hru
```

In [ ]:
# potential
pet = pd.read_csv(data_dir / "potet.csv", index_col=0)
intcpevap = pd.read_csv(data_dir / "hru_intcpevap.csv", index_col=0)
snowevap = pd.read_csv(data_dir / "snow_evap.csv", index_col=0)
impervevap = pd.read_csv(data_dir / "hru_impervevap.csv", index_col=0)
dprstevap = pd.read_csv(data_dir / "dprst_evap_hru.csv", index_col=0)

In [ ]:
# soil pet
pet["pet m/day"] = (
    pet[tag]
    - intcpevap[tag]
    - snowevap[tag]
    - impervevap[tag]
    - dprstevap[tag]
) * in2m
pet = pet.drop(tag, axis=1)
pet.columns

In [ ]:
ax = rain.plot()
pet.plot(ax=ax, label="pet")

Control volume

In [ ]:
cv = simple_soil.base.ControlVolume(
    thickness=3.0,
    max_vertical_rate=0.0305,
    horizontal_vertical_ratio=1.0,
    theta0=0.3,
    theta_wp=0.1,
    theta_fc=0.3,
    theta_sat=0.44,
    infiltration_method="green-ampt",
    soil="loam",
)
cv.infiltration_method.set_infiltration_time(1.0)

In [ ]:
for inflow, pet_rate in zip(rain["ppt m/day"], pet["pet m/day"]):
    cv.update(inflow_rate=inflow, pet_rate=pet_rate * 0.0)

In [ ]:
df = cv.get_dataframe(normalize=True)

In [ ]:
df

In [ ]:
ax = (df["rejected_inflow_L/T"] + df["inflow_L/T"]).plot()
df["rejected_inflow_L/T"].plot(ax=ax)
df["inflow_L/T"].plot(ax=ax)
ax.legend()

In [ ]:
ax = df["theta"].plot()
ax.axhline(cv.theta_fc, lw=0.5, color="0.5")
ax.axhline(cv.theta_discharge, lw=0.5, color="0.5")
ax.axhline(cv.theta_sat, lw=0.5, color="0.5")

In [ ]:
ax = df["recharge_L/T"].plot()
df["lateral_L/T"].plot(ax=ax, ls="--")
df["storage_change_L/T"].plot(ax=ax)
ax.legend()